<a href="https://colab.research.google.com/github/changdaeoh/HandsOn_DL/blob/main/sub_materials/5_GAN_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 구글드라이브와 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## DCGAN
* 네트워크는 전부 convolutional layer로 구성되며, 판별기의 pooling층(down sampling)은 large stride conv으로 대체, 생성기에서는 transposed conv를 사용

In [2]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D, Conv2DTranspose
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt
plt.style.use("seaborn-dark")

import sys

import numpy as np

conv층에서 1보다 큰 stride에 대해 padding = "same"을 적용시<br/>
output = (input + pad - kernel)/stride + 1 에서 i + p - k가 s로 나눠떨어지게 하는 최소의 p가 padding size가 됨

In [15]:
class DCGAN():
    def __init__(self, rows, cols, channels, z = 10):
        # input shape
        self.img_rows = rows
        self.img_cols = cols
        self.channels = channels
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = z

        optimizer = Adam(0.0002, 0.5)

        # 판별기 구축 및 컴파일
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss = "binary_crossentropy", optimizer = optimizer,
                                   metrics = ['accuracy'])
        #생성기 구축
        self.generator = self.build_generator()

        ''' functional API로 전체 e2e모델 구축
        위에서 판별기만 따로 compile해주고 생성기는 build만 했는데,
        아래 정의할 combined 모델의 compile이 생성기에 대한 loss, opt 컴파일임.
        '''
        # 생성기 - 노이즈를 입력받아 이미지 생성
        z = Input(shape = (self.latent_dim,))
        img = self.generator(z)
        # 판별기 - 이미지를 입력받아 진위여부 판단
        valid = self.discriminator(img)
        # 결합모델
        self.combined = Model(z, valid)
        self.combined.compile(loss = "binary_crossentropy", optimizer = optimizer)

    # 생성기 정의
    def build_generator(self):
        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation = "relu", input_dim = self.latent_dim)) # 고차원 사영
        model.add(Reshape((7, 7, 128))) # 3차원 텐서로 형상변경

        model.add(UpSampling2D()) # (14, 14, 128)
        model.add(Conv2D(128, kernel_size=3, padding = "same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))

        model.add(UpSampling2D()) # (28, 28, 128)
        model.add(Conv2D(64, kernel_size = 3, padding = "same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))

        model.add(Conv2D(self.channels, kernel_size=3, padding="same")) #  원본이미지와 같은 shape
        model.add(Activation("tanh"))

        model.summary()

        # functional API로 generator 반환
        noise = Input(shape = (self.latent_dim, ))
        img = model(noise)

        return Model(noise, img)

    # 판별기 정의
    def build_discriminator(self):
        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same")) 
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)
    
    def train(self, epochs, batch_size = 256, save_interval = 50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis = 3) # (b, w, h) -> (b, w, h, c)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):
            # ---------------------
            #  Train Discriminator
            # ---------------------
            
            # batch size개 만큼의 훈련이미지 랜덤샘플링
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # 노이즈를 샘플링하여 생성자에 전달 -> 인조 이미지 배치 생성
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # 판별기 훈련(진짜는 1, 가짜는 0)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) # 평균 loss

            # ---------------------
            #  Train Generator
            # ---------------------

            # 생성이미지에 valid label을 붙여 생성기를 훈련
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print(f"epoch{epoch} => [loss:{d_loss[0]:.2f}], [acc:{d_loss[1]*100:.2f}], [gen_loss:{g_loss:.2f}]")
            
            if epoch % save_interval == 0:
                self.save_imgs(epoch)
        
    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("./dcgan_mnist_%d.png" % epoch)
        plt.close()

In [6]:
import os

os.chdir("/content/drive/MyDrive/Colab Notebooks/results")

In [17]:
dcgan = DCGAN(28,28,1)
dcgan.train(epochs=5000, batch_size=256, save_interval=50)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
epoch0 => [loss:0.99], [acc:38.09], [gen_loss:1.07]
epoch1 => [loss:0.50], [acc:72.66], [gen_loss:0.63]
epoch2 => [loss:0.22], [acc:92.97], [gen_loss:0.32]
epoch3 => [loss:0.11], [acc:98.44], [gen_loss:0.14]
epoch4 => [loss:0.06], [acc:100.00], [gen_loss:0.04]
epoch5 => [loss:0.04], [acc:100.00], [gen_loss:0.02]
epoch6 => [loss:0.04], [acc:99.80], [gen_loss:0.02]
epoch7 => [loss:0.04], [acc:100.00], [gen_loss:0.01]
epoch8 => [loss:0.05], [acc:99.61], [gen_loss:0.01]
epoch9 => [loss:0.06], [acc:99.41], [gen_loss:0.02]
epoch10 => [loss:0.09], [acc:97.46], [gen_loss:0.02]
epoch11 => [loss:0.12], [acc:95.70], [gen_loss:0.08]
epoch12 => [loss:0.19], [acc:91.80], [gen_loss:0.20]
epoch13 => [loss:0.76], [acc:59.77], [gen_loss:0.62]
epoch14 => [loss:0.79], [acc:57.23], [gen_loss:0.41]
epoch15 => [loss:0.56], [acc:69.73], [gen_loss:0.32]
epoch16 => [loss:0.84], [acc:57.62], [gen_loss:0.41]
epoch17 => [loss:1.01], [acc:52.15], [gen_loss:0.58]
epoch18 => [loss: